In [1]:
from data.data_store import create_dataset, download_data
from feature.feature import create_batch_feature
from model.train import train_model, eval_model

import numpy as np

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [2]:
start_date = "2023-01-01"
end_date = "2024-12-31"

stock_lists = ["AAPL", "MSFT", "NVDA", "AMZN", "GOOG", "AVGO"]
for i, stock in enumerate(stock_lists[:1]):
    print(">>>>>>>>>stock: ", stock)
    df = create_dataset(stock, start_date, end_date)
    features, labels, dates = create_batch_feature(df)
    if i == 0:
        all_features, all_labels, all_dates = features, labels, dates
    else:
        all_features = np.concatenate((all_features, features), axis=0)
        all_labels = np.concatenate((all_labels, labels), axis=0)
        all_dates += dates
print("total # of data samples: ", all_features.shape[0])

>>>>>>>>>stock:  AAPL


/Users/wz/Documents/Quant/Quant-Modeling/data/data_store.py:98: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  earnings = earnings.loc[start_date:end_date]


total # of data samples:  417


In [3]:

model, criterion, test_loader, idx_test = train_model(all_features, all_labels, latent_dim=32, hidden_dim=128, epochs=100)
total_params = sum(p.numel() for p in model.parameters())
print("total # of model params: ", total_params)


Epoch [10/100], Loss: 0.4084
Epoch [20/100], Loss: 0.3909
Epoch [30/100], Loss: 0.3937
Epoch [40/100], Loss: 0.3974
Epoch [50/100], Loss: 0.4278
Epoch [60/100], Loss: 0.3370
Epoch [70/100], Loss: 0.3164
Epoch [80/100], Loss: 0.3419
Epoch [90/100], Loss: 0.3322
Epoch [100/100], Loss: 0.2926
total # of model params:  281734


In [4]:
pr_table, dates_table = eval_model(model, criterion, test_loader, idx_test, all_dates) 

Test Loss: 0.4342
           trend_5days+  trend_5days-  trend_10days+  trend_10days-  \
Precision        0.5625      0.500000       0.636364       0.375000   
Recall           0.3750      0.222222       0.368421       0.333333   

           trend_30days+  trend_30days-  
Precision       0.500000       0.333333  
Recall          0.294118       0.333333  


In [5]:
names_metrics = dates_table.columns
columns = [i for i, name in enumerate(names_metrics) if "TP" in name]
dates_table.iloc[:, columns]

,trend_5days+TP,trend_5days-TP,trend_10days+TP,trend_10days-TP,trend_30days+TP,trend_30days-TP
0,2023-03-13,2024-01-23,2023-03-13,2024-01-23,2023-03-13,2024-01-23
1,2023-03-15,2024-01-26,2023-03-15,2024-01-26,2023-03-15,2024-01-26
2,2023-05-24,2024-08-30,2023-08-21,2024-08-30,2024-05-24,None
3,2023-08-21,2024-09-03,2023-09-29,None,2024-05-29,None
4,2023-09-29,None,2024-05-24,None,2024-07-29,None
5,2024-05-24,None,2024-05-29,None,None,None
6,2024-05-29,None,2024-07-29,None,None,None
7,2024-07-29,None,None,None,None,None
8,2024-08-15,None,None,None,None,None
9,None,None,None,None,None,None


In [6]:

label_names = pr_table.columns
labels_table = pd.DataFrame(
    data=labels,
    index=dates,
    columns=label_names
)
start_date = dates_table.iloc[0, 12]
end_date = dates_table.iloc[5, 12]
labels_table.loc[start_date:end_date:10]

NameError: name 'pd' is not defined